In [1]:
# Install required packages (run once)
%pip install -q vnstock pandas numpy tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm

# Use vnstock3 unified API
try:
    from vnstock3 import Finance, Listing
except Exception as e:
    raise RuntimeError('vnstock3 not available. Please install vnstock3 in the venv.')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 160)

print('Environment ready with vnstock3.')


/Users/nguyenhuycuong/Crownwell Stock/.venv/lib/python3.13/site-packages/vnai/scope/profile.py:562: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Thư viện `vnstock3` đã được hợp nhất với tên gọi `vnstock`. Phiên bản mới nhất 3.2.6 đã có mặt.Vui lòng chuyển đổi gói thư viện thành `vnstock` để luôn nhận bản cập nhật với câu lệnh : `pip install vnstock --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 3.2.1

Environment ready with vnstock3.


## Criteria mapping (draft)

This notebook implements quantitative filters based on your table:

- Market position: proxy by market cap rank within industry (Top 3) — requires sector mapping.
- Growth: revenue and profit CAGR over 3 years >= thresholds.
- Profitability: ROE, ROA, gross margin.
- Valuation: PEG <= 1.5, EV/EBITDA <= 10, P/B <= 2 (as proxies based on your sheet).
- Liquidity: Avg trading value >= 1e9 VND/day (adjust as needed).
- Ownership/float: Free float >= 40% when available.

Qualitative items (brand strength, moat, management, ESG, strategy, new trends) are left as manual review columns.


In [3]:
# Helper: fetch tickers, fundamentals using vnstock3

def get_all_tickers(exchange: str = 'ALL') -> pd.DataFrame:
    boards = ['HOSE', 'HNX', 'UPCOM'] if exchange == 'ALL' else [exchange]
    frames = []
    ls = Listing()
    for b in boards:
        try:
            df = ls.symbols_by_exchange(exchange=b)
            df['exchange'] = b
            frames.append(df[['symbol','exchange']])
        except Exception:
            pass
    if not frames:
        raise RuntimeError('Could not retrieve tickers')
    return pd.concat(frames, ignore_index=True)


def get_ratios(symbol: str) -> pd.DataFrame:
    try:
        df = Finance(symbol=symbol).ratio(period='year')
        df['symbol'] = symbol
        return df.reset_index().rename(columns={'period':'year'})
    except Exception:
        return pd.DataFrame()


def get_income(symbol: str) -> pd.DataFrame:
    try:
        df = Finance(symbol=symbol).income_statement(period='year')
        df['symbol'] = symbol
        return df.reset_index().rename(columns={'period':'year'})
    except Exception:
        return pd.DataFrame()


def get_overview(symbol: str) -> dict:
    return {'symbol': symbol}



In [4]:
# Config: thresholds (adjust if needed)
CRITERIA = {
    'market_rank_top_n': 3,
    'min_revenue_cagr_3y': 0.05,   # 5%
    'min_profit_cagr_3y': 0.10,    # 10%
    'min_roe': 0.10,               # 10%
    'min_roa': 0.03,               # 3%
    'max_peg': 1.5,
    'max_ev_ebitda': 12.0,
    'max_pb': 3.0,
    'min_avg_trading_value_vnd': 500_000_000,  # 0.5B VND/day
    'min_free_float': 0.30
}
CRITERIA


{'market_rank_top_n': 3,
 'min_revenue_cagr_3y': 0.05,
 'min_profit_cagr_3y': 0.1,
 'min_roe': 0.1,
 'min_roa': 0.03,
 'max_peg': 1.5,
 'max_ev_ebitda': 12.0,
 'max_pb': 3.0,
 'min_avg_trading_value_vnd': 500000000,
 'min_free_float': 0.3}

In [5]:
# Fetch tickers and fundamentals using vnstock3
boards = ['HOSE', 'HNX', 'UPCOM']
try:
    ls = Listing()
    tickers = []
    for b in boards:
        dfb = ls.symbols_by_exchange(exchange=b)
        tickers.extend(sorted(dfb['symbol'].unique().tolist()))
    tickers = sorted(list(set(tickers)))
except Exception:
    tickers = ['VIC','VHM','VNM','HPG','VCB','ACB','MWG','FPT','SSI','GAS']

len(tickers), tickers[:10]


(10, ['VIC', 'VHM', 'VNM', 'HPG', 'VCB', 'ACB', 'MWG', 'FPT', 'SSI', 'GAS'])

In [6]:
# Download yearly income statement and ratios for each ticker (can take time)
ratio_list = []
inc_list = []
for sym in tqdm(tickers, desc='Downloading fundamentals'):
    r = get_ratios(sym)
    if not r.empty:
        ratio_list.append(r)
    i = get_income(sym)
    if not i.empty:
        inc_list.append(i)

ratios = pd.concat(ratio_list, ignore_index=True) if ratio_list else pd.DataFrame()
income = pd.concat(inc_list, ignore_index=True) if inc_list else pd.DataFrame()

ratios.head(), income.head()


(   year  price_to_earning  price_to_book  value_before_ebitda  dividend    roe    roa  days_receivable  days_inventory  days_payable  ebit_on_interest  \
 0  2024              13.1            1.1                 28.4       NaN  0.095  0.016           -161.0           233.0          80.0              -0.3   
 1  2023              79.7            1.5                 23.9       NaN  0.019  0.003            -83.0           253.0          99.0              -0.1   
 2  2022              23.6            1.9                 35.0       NaN  0.083  0.017             -1.0           312.0          76.0              -1.0   
 3  2021            -145.8            3.6                 26.1       NaN -0.028 -0.006             17.0           225.0          88.0               0.3   
 4  2020              67.8            4.7                 91.8       NaN  0.070  0.013             21.0           287.0          92.0              -0.3   
 
    earning_per_share  book_value_per_share  equity_on_total_asset  

In [7]:
# Compute derived metrics: CAGR, PEG, EV/EBITDA proxies

def compute_cagr(series: pd.Series, years: int = 3) -> float:
    series = series.dropna().sort_index()
    if len(series) < years + 1:
        return np.nan
    start = series.iloc[-(years+1)]
    end = series.iloc[-1]
    if start <= 0:
        return np.nan
    return (end / start) ** (1/years) - 1

# Prepare income by symbol-year pivot using vnstock3 columns
if not income.empty:
    inc = income.copy()
    inc.columns = [c.lower() for c in inc.columns]
    year_col = 'year'
    revenue_col = 'revenue'
    profit_col = 'post_tax_profit'  # vnstock3 income_statement column

    # Optional yearly growth columns provided by vnstock3
    rev_growth_col = 'year_revenue_growth' if 'year_revenue_growth' in inc.columns else None
    profit_growth_col = 'year_share_holder_income_growth' if 'year_share_holder_income_growth' in inc.columns else None

    cagr_rows = []
    for sym, g in inc.groupby('symbol'):
        try:
            g = g.sort_values(by=year_col)
            rev_series = g.set_index(year_col)[revenue_col]
            prof_series = g.set_index(year_col)[profit_col]
            rev_cagr = compute_cagr(rev_series)
            prof_cagr = compute_cagr(prof_series)
            # Fallback: average of last 3 yearly growth values if CAGR NaN
            if pd.isna(rev_cagr) and rev_growth_col:
                rev_cagr = g[rev_growth_col].tail(3).mean()
            if pd.isna(prof_cagr) and profit_growth_col:
                prof_cagr = g[profit_growth_col].tail(3).mean()
        except Exception:
            rev_cagr = np.nan
            prof_cagr = np.nan
        cagr_rows.append({'symbol': sym, 'rev_cagr_3y': rev_cagr, 'profit_cagr_3y': prof_cagr})
    cagr_df = pd.DataFrame(cagr_rows)
else:
    cagr_df = pd.DataFrame(columns=['symbol','rev_cagr_3y','profit_cagr_3y'])

# Ratios normalization using vnstock3 columns
if not ratios.empty:
    r = ratios.copy()
    r.columns = [c.lower() for c in r.columns]
    latest = r.sort_values(['symbol','year']).groupby('symbol').tail(1)
    # vnstock3 ratio columns
    col_map = {
        'price_to_earning': 'pe',
        'price_to_book': 'pb',
        'return_on_equity': 'roe',
        'return_on_asset': 'roa',
    }
    keep = ['symbol'] + [c for c in col_map.keys() if c in latest.columns]
    latest = latest[keep].rename(columns=col_map)
    for c in ['roe','roa','pe','pb']:
        if c not in latest.columns:
            latest[c] = np.nan
else:
    latest = pd.DataFrame(columns=['symbol','roe','roa','pe','pb'])

metrics = latest.merge(cagr_df, on='symbol', how='left')
metrics.head()


,symbol,pe,pb,roe,roa,rev_cagr_3y,profit_cagr_3y
0,ACB,6.6,1.3,NaN,NaN,0.136294,0.204707
1,FPT,28.5,7.5,NaN,NaN,0.207956,0.207907
2,GAS,14.9,2.6,NaN,NaN,0.094482,0.061577
3,HPG,14.2,1.5,NaN,NaN,-0.024713,-0.296484
4,MWG,23.9,3.2,NaN,NaN,0.029953,-0.086747


In [8]:
# Apply filters based on CRITERIA
m = metrics.copy()
# Convert percentage ratios if in percent
for col in ['roe','roa']:
    if col in m.columns and m[col].notna().any():
        if m[col].max() > 1:
            m[col] = m[col] / 100.0

# PEG only if growth available
m['peg'] = m.get('pe') / (m.get('profit_cagr_3y')) if ('pe' in m.columns and 'profit_cagr_3y' in m.columns) else np.nan

# Growth: allow either revenue or profit to meet thresholds
growth_ok = (
    (m.get('rev_cagr_3y').fillna(-1) >= CRITERIA['min_revenue_cagr_3y']) |
    (m.get('profit_cagr_3y').fillna(-1) >= CRITERIA['min_profit_cagr_3y'])
)

# Profitability: at least one of ROE/ROA meets threshold when available
roe_ok = (m.get('roe').fillna(-1) >= CRITERIA['min_roe'])
roa_ok = (m.get('roa').fillna(-1) >= CRITERIA['min_roa'])
profitability_ok = roe_ok | roa_ok | (m.get('roe').isna() & m.get('roa').isna())

# Valuation: PB optional, PEG optional
pb_ok = (('pb' not in m.columns) | m['pb'].isna() | (m['pb'] <= CRITERIA['max_pb']))
peg_ok = (('peg' not in m.columns) | pd.isna(m['peg']) | (m['peg'] <= CRITERIA['max_peg']))

cond = growth_ok & profitability_ok & pb_ok & peg_ok

filtered = m[cond].sort_values(by=[c for c in ['profit_cagr_3y','rev_cagr_3y','roe'] if c in m.columns], ascending=False).reset_index(drop=True)
filtered.head(20)


,symbol,pe,pb,roe,roa,rev_cagr_3y,profit_cagr_3y,peg
0,VHM,5.2,0.8,NaN,NaN,0.063837,-0.034329,-151.476037


In [9]:
# Export and add qualitative placeholders
output_path = 'vn_screener_results.csv'
q_cols = ['brand_strength','moat','management_quality','esg_flag','strategy_alignment','new_trend_exposure']
res = filtered.copy()
for c in q_cols:
    res[c] = ''
res.to_csv(output_path, index=False)
print(f'Saved {len(res)} rows to {output_path}')
res.head()


Saved 1 rows to vn_screener_results.csv


,symbol,pe,pb,roe,roa,rev_cagr_3y,profit_cagr_3y,peg,brand_strength,moat,management_quality,esg_flag,strategy_alignment,new_trend_exposure
0,VHM,5.2,0.8,NaN,NaN,0.063837,-0.034329,-151.476037,,,,,,
